In [1]:
pip install MLB-statsAPI

Note: you may need to restart the kernel to use updated packages.


In [192]:
import streamlit as st
import pickle
import pandas as pd
from datetime import date, timedelta
import statsapi
import joblib


In [193]:
model = joblib.load('best_pitch_model.pkl')

In [194]:
schedule = statsapi.schedule(start_date=date.today(), end_date=date.today(), team="135",sportId=1)
if schedule:
    game_id = schedule[0]['game_id']

In [195]:
def get_padres_pitchers():
    team_id = 135  # San Diego Padres
    roster = statsapi.get('team_roster', {'teamId': team_id})
    
    pitchers = []
    for player in roster.get('roster', []):
        if player['position']['type'] == 'Pitcher':
            pitchers.append({
                'id': player['person']['id'],
                'name': player['person']['fullName']
            })
    return pitchers

pitchers = get_padres_pitchers()
pitcher_ids = [p['id'] for p in pitchers]

In [196]:
live_feed = statsapi.get('game_playByPlay', {'gamePk': game_id})
all_plays = live_feed.get('allPlays', [])

In [223]:
live_feed = statsapi.get('game_playByPlay', {'gamePk': game_id})
all_plays = live_feed.get('allPlays', [])
current_play = all_plays[len(all_plays)-1]
play_events = current_play.get('playEvents', [])
if play_events:
    prev_pitch_type = play_events[-1].get('details', {}).get('type', {}).get('code', 'firstPitch')
else:
    prev_pitch_type = 'firstPitch'
print(prev_pitch_type)
current_play['count']

CH


{'balls': 1, 'strikes': 2, 'outs': 1}

In [203]:
abs(current_play['result'].get('home_score') - current_play['result'].get('away_score'))

TypeError: unsupported operand type(s) for -: 'NoneType' and 'NoneType'

In [206]:
def get_next_padres_pitch_context(game_id, padres_pitcher_ids):
    live_feed = statsapi.get('game_playByPlay', {'gamePk': game_id})
    all_plays = live_feed.get('allPlays', [])
    current_play = all_plays[-1]

    
    features = {
        'pitcher': current_play['matchup'].get('pitcher', {}).get('id', 'None'),
        'batter': current_play['matchup'].get('batter', {}).get('id', 'None'),
        'on_1b': int('postOnFirst' in current_play['matchup']),
        'on_2b': int('postOnSecond' in current_play['matchup']),
        'on_3b': int('postOnThird' in current_play['matchup']),
        'prev_pitch_type': current_play['playEvents'][len(current_play['playEvents'])-1]['details']['type'].get('code', 'firstPitch'),
        'inning':  current_play['about'].get('inning'),
        'balls':  current_play['count'].get('balls'),
        'strikes': current_play['count'].get('strikes'),
        'outs_when_up': current_play['count'].get('outs'),
        'score_diff': abs(current_play['result'].get('homeScore') - current_play['result'].get('awayScore'))
                }

    return features


In [207]:
data = get_next_padres_pitch_context(game_id, pitcher_ids)
input_df = pd.DataFrame([data])
input_df

,pitcher,batter,on_1b,on_2b,on_3b,prev_pitch_type,inning,balls,strikes,outs_when_up,score_diff
0,571760,650333,0,0,0,FF,3,2,0,2,2


In [80]:
encode_dict = {0: 'CH',
 1: 'CU',
 2: 'FC',
 3: 'FF',
 4: 'FS',
 5: 'KC',
 6: 'SI',
 7: 'SL',
 8: 'ST'}

In [84]:
code_to_name = {'ST': 'Sweeper',
 'CH': 'Changeup',
 'FF': 'Four Seam Fastball',
 'SI': 'Sinker',
 'SL': 'Slider',
 'FC': 'Cutter',
 'CU': 'Curveball',
 'KC': 'Knuckleball',
 'FS': 'Split-finger'}

In [87]:
prediction = model.predict(input_df)
pitch_type = encode_dict.get(prediction[0])
name_of_pitch = code_to_name.get(pitch_type)
print("Predicted pitch type:", name_of_pitch)

Predicted pitch type: Cutter


In [88]:
st.title("Pitch Type Predictor")

if st.button("Predict Pitch Type"):
    prediction = model.predict(input_df)
    pitch_type = encode_dict.get(prediction[0])
    name_of_pitch = code_to_name.get(pitch_type)
    st.write(f"Predicted pitch type: {name_of_pitch}")

2025-05-21 23:25:08.432 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-21 23:25:08.434 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-21 23:25:08.435 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-21 23:25:08.436 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-21 23:25:08.437 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-21 23:25:08.439 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-21 23:25:08.440 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [41]:
import requests
import pandas as pd

game_id = game_id  # replace with your gamePk
url = f"https://statsapi.mlb.com/api/v1.1/game/{game_id}/feed/live"
response = requests.get(url)
live_feed = response.json()

teams = live_feed["gameData"]["teams"]
home_team_id = teams["home"]["id"]
away_team_id = teams["away"]["id"]

all_plays = live_feed.get("liveData", {}).get("plays", {}).get("allPlays", [])
data = []

for play in all_plays:
    matchup = play.get("matchup", {})
    result = play.get("result", {})
    count = play.get("count", {})
    about = play.get("about", {})
    runners = play.get("runners", [])
    pitch_events = play.get("pitchEvents", [])

    if not pitch_events:
        continue

    # Determine last pitch type properly
    if len(pitch_events) == 1:
        # This is the first pitch of the at-bat
        last_pitch_type = "firstpitch"
    else:
        # Get the pitch type of the pitch *before* the current pitch
        last_pitch_type = None
        for event in reversed(pitch_events[:-1]):
            pt = event.get("details", {}).get("type", {}).get("code")
            if pt:
                last_pitch_type = pt
                break
        if last_pitch_type is None:
            last_pitch_type = "firstpitch"

    # Baserunners
    on_1b = 0
    on_2b = 0
    for runner in runners:
        base = runner.get("movement", {}).get("start")
        if base == "1B":
            on_1b = 1
        elif base == "2B":
            on_2b = 1

    # Score differential
    home_score = about.get("homeScore", 0)
    away_score = about.get("awayScore", 0)
    pitcher_team_id = matchup.get("pitcher", {}).get("currentTeam", {}).get("id")
    score_diff = (home_score - away_score) if pitcher_team_id == home_team_id else (away_score - home_score)

    data.append({
        "pitcher_id": matchup.get("pitcher", {}).get("id"),
        "batter_id": matchup.get("batter", {}).get("id"),
        "inning": about.get("inning"),
        "balls": count.get("balls"),
        "strikes": count.get("strikes"),
        "outs": count.get("outs"),
        "last_pitch_type": last_pitch_type,
        "score_diff": score_diff,
        "on_1b": on_1b,
        "on_2b": on_2b,
        "event": result.get("event")
    })

df = pd.DataFrame(data)
print(df.head())


Empty DataFrame
Columns: []
Index: []


In [42]:
df

""


In [46]:
https://statsapi.mlb.com/api/v1/eventTypes

SyntaxError: invalid syntax (1101770160.py, line 1)

In [59]:
url = f"https://statsapi.mlb.com/api/v1/eventTypes"
response = requests.get(url)
live_feed = response.json()

NameError: name 'code' is not defined